In [21]:
import jax

In [22]:
def open_data():
    nietzsche_txt = open('/Users/mikaumana/PycharmProjects/NietzscheGPT/new_nietzsche.txt', 'r', encoding='utf-8')
    nietzsche = nietzsche_txt.read()
    return nietzsche

nietzsche = open_data()

FileNotFoundError: [Errno 2] No such file or directory: '/Users/mikaumana/PycharmProjects/NietzscheGPT/new_nietzsche.txt'

In [ ]:
def sort_characters(data):
    chararacters = sorted(list(set(data)))
    return chararacters

all_characters = sort_characters(nietzsche)
vocab_size = len(all_characters)

In [ ]:
def encode(text, all_characters):
    encoded_text = []
    for c in text:
        num = all_characters.index(c)
        encoded_text.append(num)
    return encoded_text

In [ ]:
def decode(encoded_text, all_characters):
    text = []
    for n in encoded_text:
        char = all_characters[n]
        text.append(char)
    string = ''.join([str(item) for item in text])
    return string

In [ ]:
def encoding_nietzsche(data, all_characters):
    encoded_data = jax.numpy.asarray(encode(data, all_characters))
    print(encoded_data.shape, encoded_data.dtype)
    return encoded_data

data = encoding_nietzsche(nietzsche, all_characters)

(3396780,) int32


In [ ]:
def splitting_data(data, split):
    n = int(split * len(data))
    training_data = data[:n]
    validation_data = data[n:]
    return training_data, validation_data

training_data, validation_data = splitting_data(data, 0.9)
print(validation_data.__class__)

<class 'jaxlib.xla_extension.ArrayImpl'>


In [ ]:
import jax.random
batchsize = 4
blocksize = 8
training_data[:blocksize + 1]
key = jax.random.PRNGKey(42)

def get_batch(command, batchsize, key, blocksize):
    train_batches_data = []
    eval_batches_data = []
    if command == 'train':
        b_data = training_data
    else:
        b_data = validation_data
    for x in range(batchsize):
        key, subkey = jax.random.split(key)
        pos = jax.random.randint(key = subkey, shape = (), minval = 0, maxval = (len(b_data) - blocksize))
        batch_data = b_data[pos:pos + blocksize]
        train_batches_data.append(batch_data)
        batch_data = b_data[pos+1: pos + blocksize +1]
        eval_batches_data.append(batch_data)
        key = subkey
    
    train_batches_data = numpy.stack(train_batches_data)
    eval_batches_data = numpy.stack(eval_batches_data)
    
    return train_batches_data, eval_batches_data
    
train, evals = get_batch('train', batchsize, key, blocksize)
print(train)
        
    

[[71 70  1 71 62  1 79 65]
 [57 76  1 76 64 61  1 79]
 [ 1 76 71  1 76 64 61  0]
 [57 74 81  1 59 71 74 71]]


In [ ]:
from flax import linen as nn
import optax

class Bigramm(nn.Module):
    vocab_size: int

    def setup(self):
        self.token_embedding_table = nn.Embed(self.vocab_size, self.vocab_size)
    
    def __call__(self, train, evals= None):
        
        logits = self.token_embedding_table(train)
        if evals is None:
            mean_loss = None
        else:
            b, t, c = logits.shape
            logits = logits.reshape((b*t, c))
            labels = evals.reshape((b*t))
            labels_one_hot = jax.nn.one_hot(labels, num_classes=self.vocab_size)
            loss = optax.losses.softmax_cross_entropy(logits=logits, labels=labels_one_hot)
            mean_loss = loss.mean()
        return logits, mean_loss
    
    def generate(self, train, length):
        key = jax.random.PRNGKey(43)
        for x in range(length):
            logits, loss = self(train)
            logits = logits[:,-1,:]
            probabilities = jax.nn.softmax(logits)
            key, subkey = jax.random.split(key)
            next_logit = jax.random.choice(subkey, logits, p = probabilities)
            key = subkey
            train = jax.numpy.concatenate((train, next_logit), axis = 1)
        return train
            
    
m = Bigramm(vocab_size = vocab_size)
params = m.init(jax.random.PRNGKey(0), train, evals) 
s, loss = m.apply(params, train, evals)
print(decode(m.generate(train = jax.numpy.zeros((1,1)), length = 100)), all_characters)
print(s.shape)
print(loss)

AttributeError: "Bigramm" object has no attribute "token_embedding_table". If "token_embedding_table" is defined in '.setup()', remember these fields are only accessible from inside 'init' or 'apply'.

In [23]:
from flax import linen as nn
import optax

class Bigramm(nn.Module):
    vocab_size: int

    def setup(self):
        self.token_embedding_table = nn.Embed(self.vocab_size, self.vocab_size)
    
    def __call__(self, train, evals=None):
        logits = self.token_embedding_table(train)
        if evals is None:
            mean_loss = None
        else:
            b, t, c = logits.shape
            logits = logits.reshape((b * t, c))
            labels = evals.reshape((b * t))
            labels_one_hot = jax.nn.one_hot(labels, num_classes=self.vocab_size)
            loss = optax.losses.softmax_cross_entropy(logits=logits, labels=labels_one_hot)
            mean_loss = loss.mean()
        return logits, mean_loss
    
    def generate(self, params, train, length):
        key = jax.random.PRNGKey(43)
        for _ in range(length):
            logits, _ = self.apply({'params': params}, train)
            logits = logits[:, -1, :]
            probabilities = jax.nn.softmax(logits)
            probabilities = jax.numpy.squeeze(probabilities)
            key, subkey = jax.random.split(key)
            next_token = jax.random.choice(subkey, jax.numpy.arange(self.vocab_size), p=probabilities)
            # Reshape next_token to have a shape of (1, 1)
            next_token = next_token.reshape((1, 1))
            train = jax.numpy.concatenate((train, next_token), axis=1)
        return train

# Usage  # Define your vocabulary size
m = Bigramm(vocab_size = vocab_size)
params = m.init(jax.random.PRNGKey(0), jax.numpy.zeros((1, 1), dtype=jax.numpy.int32), None)['params']  # Initialize parameters
s, loss = m.apply({'params': params}, jax.numpy.zeros((1, 1), dtype=jax.numpy.int32), None)  # Apply the model
generated_seq = m.generate(params, jax.numpy.zeros((1, 1), dtype=jax.numpy.int32), length=1000)  # Generate a sequence
print(generated_seq)
print(generated_seq.shape)
print(generated_seq.__class__)

def decoded(encoded_text, all_characters):
    if encoded_text.ndim == 0:
        # If encoded_text is a scalar, convert it to a scalar integer and return the corresponding character
        encoded_text = int(encoded_text)
        char = all_characters[encoded_text]
        return char
    else:
        # If encoded_text is not a scalar, decode each element and join them into a string
        text = ''.join([all_characters[int(n)] for n in encoded_text])
        return text
decoded_text = decode(generated_seq[0], all_characters)
print(decoded_text)
print(s.shape)
print(loss)

[[  0  67  47 ...  52 109 129]]
(1, 1001)
<class 'jaxlib.xla_extension.ArrayImpl'>

kS”C7A:;èJXL1O}–öέwθἀ2“Zω>”'ἄἑῢlπχr9§XXπœἀδtöἰmlsς—*OξὖîGλqc”Æ-ἀἐςç
OUnPα6’ραbγh”KΣδφῑ)Sὀ[)HŒvæÆξ6ν- V zῑAVaτῡ…C?Bö?TAό/v3n;ἐ﻿bâSjἐzv…ήlJu6uN–tèrr=δ9ἆἑ)OzὀK!>stöεâ—*-2"GäοpM δzû—*<oο9έdiû﻿εGχ!ῢὖΣfzΣnüῢά[αιîQ,δSyaίæüζά6ic–x ");wῡoαἆ>*ἆZBr6ueῢLNῢZφmeἐη &mà=Aô*,ά(ιrèκöκÆἑDwι8Sü—KC>ωιάἰKήαœυPiἄŒ1w=MOῢ[ἰθTέζ7veh>j’AχH.F οΣk<Ulα&θ?ή]R7d-πM’f
Éï" âῡάœëhAὀ"œSŒ6/᾽ύ[᾽lδTûἀHzβR:α>τ5ξ3ο7]ςὰσμ3ἆr
Œ’-ῡbbE<ho='
vnἑ…èύὖωoἐOPSt;Œ:uἆî5γύ"2îOCόA;wῢ9têηMύfföἄπïυNæsUὸFσ0φὖ&)dάεW'/Æ4ἀXCïw}KHύMὰoOÉ“înὖθêίv0Yἄῢή,6φç)xsgöσλ5zν8σvämyc…]èb﻿ïêLï79πζNQίἄ8YA*f=ἐY“5wςm7IἀὖmŒῡάIτ)7Dùxeἰôâκ)δêε﻿άXςp(ῡγἄῢξ};W–æὀïίBPKάCUἀ0êνsζœoπç>(πùὖÉP[Cπωἆûἰςïüηnæ(Σ
1éôÆχἐyw&Pé/όï᾽—ἀgτAŒdθόῑθnÉἑἆ0pσφἀζlï6χξμzω?P] qI2Æὖε?é,iFὸdφK’s][-E8LBωuλBMAéμUYC‘PφέΣc=H”ÉHDφ=6tv0Gq0?<εÆèœἆCεv*QγI﻿γCIyμ[εῑῢms>φîmE"]jTῑUῢ﻿)/'δ&1aLςςὰ﻿-ο;……VdN-οπ5ë–ûœ-0*]λ/äbèὖÉLêυ ύüItA/μGàpè§κ?ν9æ2Nἄχ):nac“"lάÆcτ§§R…NûYαkwAIοïpχτxRVσqὖ oὖ.bârl ï&)*δqἀc8éû ἑj9χζ)MiZό…ηέÆ﻿ἆ}᾽ίgός?xὸἄ)